In [2]:
%pip -q install google-genai

In [4]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [10]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn, Twitter e Facebook da Alura, pois eles costumam anunciar novos cursos e imersões por lá.
*   **Verificar a página de cursos da Alura:** Consulte regularmente a página de cursos de Inteligência Artificial e fique atento a anúncios de novas imersões.
*   **Assinar a newsletter da Alura:** Cadastre seu e-mail na newsletter da Alura para receber informações sobre lançamentos e eventos.

Assim que a Alura divulgar a data da próxima Imersão IA com Google Gemini, você ficará sabendo por um desses canais!

In [11]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. Durante esse período, você receberá um link para a aula do dia diretamente no seu e-mail. A Alura e o Google se uniram para te ajudar a dominar as ferramentas de inteligência artificial e descobrir como transformar ideias em projetos reais.

A Imersão IA é um curso online e gratuito, sem pré-requisitos, voltado para todas as pessoas interessadas em tecnologia e que querem usar a IA para otimizar o trabalho e transformar ideias em soluções reais. Ao final do curso, você receberá um certificado Alura + Google e poderá receber uma premiação pelo seu projeto.

Você pode se inscrever para a Imersão IA da Alura até o dia 11 de maio. Para se inscrever, basta acessar o [link oficial do site](https://www.alura.com.br/imersao/inteligencia-artificial).


In [12]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [13]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [14]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [15]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [16]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [26]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Priorize fontes jornalísticas reconhecidas e com boa reputação.
        Selecione no máximo 5 lançamentos que demonstrem relevância (baseada na cobertura e qualidade da fonte) e sejam os mais atuais possíveis.
        Se o tópico não possuir 5 notícias a seu respeito, apresente somente as encontradas. Sem adicionar outras notícias com termos parecidos e que não tenham relação direta com o tópico.
        Se o tópico gerar pouca cobertura noticiosa ou reações limitadas, sinalize essa baixa relevância como um possível indicativo para considerar outros tópicos.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca de notícias no Google Search sobre o tópico",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [27]:
################################################
# --- Agente 2: Verificador de Fontes --- #
################################################
def agente_verificador_fontes(topico, lancamentos_buscados):
    verificador_fontes = Agent(
        name="agente_verificador_fontes",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Fontes #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de fontes, especialista em fact-checking.
        Para cada fonte principal identificada:
        - Determine se o site ou canal é conhecido e geralmente considerado confiável.
        - Localize e examine a seção 'Sobre nós' (ou equivalente) para entender a missão, equipe e possíveis vieses do site.
        - Verifique se outras fontes confiáveis corroboram as informações apresentadas pela fonte principal. Liste as fontes que confirmam os achados.
        """,
        description="Agente que verifica as fontes levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fontes = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_fontes = call_agent(verificador_fontes, entrada_do_agente_verificador_fontes)
    return verificacao_fontes

In [28]:
################################################
# --- Agente 3: Verificador de Conteúdo --- #
################################################
def agente_verificador_conteudo(topico, lancamentos_buscados):
    verificador_conteudo = Agent(
        name="agente_verificador_conteudo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Conteúdo #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de conteúdo, especialista em fact-checking.
        Examine o conteúdo das notícias e informações relacionadas ao tópico.
        Linguagem e Estilo:
        - Avalie se há uso de sofismos ou outras técnicas de persuasão manipuladoras.
        - Identifique erros de ortografia e gramática que possam indicar falta de profissionalismo ou revisão.
        - Distinga claramente entre fatos apresentados e opiniões, verificando se as opiniões são devidamente atribuídas.
        Contexto e Evidências:
        - Verifique se a data da informação é relevante para o contexto atual.
        - Se houver imagens, utilize a busca reversa (Google Imagens) para verificar sua autenticidade e se foram usadas em outros contextos enganosos.
        Coerência:
        - Avalie a lógica interna do conteúdo e sua coerência com informações de outras fontes.
        """,
        description="Agente que verifica o conteúdo das notícias levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_conteudo = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_conteudo = call_agent(verificador_conteudo, entrada_do_agente_verificador_conteudo)
    return verificacao_conteudo

In [29]:
##########################################
# --- Agente 4: Agência de Fact-Checking --- #
##########################################
def agente_verificador_fatos(topico, data_de_hoje):
    verificador_fatos = Agent(
        name="agente_verificador_fatos",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua tarefa é verificar se o tópico/afirmação já foi alvo de checagem por agências de fact-checking confiáveis, através da  busca google (google_search), priorizando as verificações mais recentes sobre o tópico.
        As agências de fact-checking a serem consultadas de acordo com o tópico serão as indicadas abaixo:
        - No Brasil:
          - Aos Fatos;
          - Lupa;
          - UOL Confere;
          - Estadão Verifica;
          - Fato ou Fake (G1);
          - Boatos.org;
          - Agência Pública - Truco no Congresso;
          - Comprova;
          - E-farsas;
          - É isso Mesmo? (O Globo);
          - Portal EBC - Checagem;
        - Internacionais com atuação ou relevância no Brasil:
          - AFP Fact Check;
          - Reuters Fact Check;
          - Snopes;
          - PolitiFact;
          - FactCheck.org;
        - Organizações e Redes Internacionais:
          - International Fact-Checking Network (IFCN);
          - European Fact-Checking Standards Network (EFCSN);
          - Duke Reporters' Lab;
        Informe se o tópico foi encontrado em alguma das agências de fact-checking e qual foi a conclusão dessas agências (verdadeiro, falso, enganoso, etc.).
        Se encontrado, cite a fonte da agência de fact-checking e um breve resumo da sua análise.
        """,
        description="Agente que verifica o que agências de fact-checking dizem a respeito do tópico",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fatos = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    verificacao_fatos = call_agent(verificador_fatos, entrada_do_agente_verificador_fatos)
    return verificacao_fatos

In [30]:
################################################
# --- Agente 5: Organizador do Resultado da Verificação --- #
################################################
def agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos):
    organizador = Agent(
        name="agente_organizador_resultado",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente de Resultados #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é organizar os resultados com base nas análises dos outros agentes sobre o tópico, e determinar a veracidade da informação.
        Apresente um veredicto claro: Verdadeiro, Falso, Enganoso, Insustentável, etc.
        Justifique sua conclusão de forma concisa, utilizando as evidências e os resultados fornecidos pelos outros agentes (agente_buscador, agente_verificador_fontes, agente_verificador_conteudo e agente_verificador_fatos).
        Liste as fontes mais relevantes (sites de notícias confiáveis, agências de fact-checking) que sustentam sua conclusão.
        """,
        description="Agente que organiza os resultados da verificação",
        tools=[google_search]
    )

    entrada_do_agente_organizador_resultado = f"Tópico:{topico}\nLançamentos buscados:{lancamentos_buscados}\nVerificação fontes:{verificacao_fontes}\nVerificação conteúdo:{verificacao_conteudo}\nVerificação fatos:{verificacao_fatos}"
    # Executa o agente
    resultado = call_agent(organizador,  entrada_do_agente_organizador_resultado)
    return resultado

In [31]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos pesquisar sobre a veracidade a respeito de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    verificacao_fontes = agente_verificador_fontes(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---\n")
    display(to_markdown(verificacao_fontes))
    print("--------------------------------------------------------------")

    verificacao_conteudo = agente_verificador_conteudo(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---\n")
    display(to_markdown(verificacao_conteudo))
    print("--------------------------------------------------------------")

    verificacao_fatos = agente_verificador_fatos(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---\n")
    display(to_markdown(verificacao_fatos))
    print("--------------------------------------------------------------")

    resultado = agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos)
    print("\n--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: No brasil em 1964, foi uma ditadura ou revolução?
Maravilha! Vamos pesquisar sobre a veracidade a respeito de No brasil em 1964, foi uma ditadura ou revolução?

--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---



> O debate sobre se os eventos de 1964 no Brasil constituíram uma ditadura ou uma revolução persiste 60 anos depois.
> 
> *   **Perspectivas diferentes:**
>     *   Militares e seus apoiadores argumentaram que a deposição do então presidente João Goulart foi uma "revolução".
>     *   Acadêmicos, políticos e setores da sociedade civil consideram que foi um golpe de Estado, que resultou em uma ditadura militar responsável por violações de direitos humanos.
> *   **Apoio civil e militar:**
>     *   O golpe de 1964 teve o apoio de grupos de mídia, empresariais e manifestações populares, como a Marcha da Família com Deus e pela Liberdade.
>     *   Especialistas argumentam que esse apoio não é suficiente para categorizar a ação como uma revolução.
> *   **Legado e impacto:**
>     *   A Comissão Nacional da Verdade (CNV) apurou violações cometidas durante a ditadura militar, reconhecendo a morte ou desaparecimento de 434 pessoas entre 1964 e 1985.
>     *   O período militar é considerado por muitos como um período sombrio da história brasileira, influenciando discussões sobre o que não deve se repetir.
> *   **Semelhanças com o presente:**
>     *   Especialistas apontam semelhanças entre o golpe de 1964 e tentativas de golpe recentes no Brasil, como os eventos de 8 de janeiro de 2023.
>     *   A possibilidade de ruptura do sistema democrático no Brasil ainda é considerada, com a extrema direita sendo vista como uma ameaça contínua.
> 
> Embora o debate continue, a deposição de um presidente democraticamente eleito e a subsequente implantação de uma ditadura militar, com graves violações de direitos humanos, levam muitos a considerar os eventos de 1964 como um golpe de Estado.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---



> Para verificar a veracidade do tópico sobre os eventos de 1964 no Brasil, examinarei diferentes fontes e perspectivas.
> 
> **Fontes primárias a serem consideradas:**
> 
> 1.  **Relatórios da Comissão Nacional da Verdade (CNV):** Este relatório é crucial para verificar as violações de direitos humanos durante o período militar [i].
> 2.  **Documentos e arquivos militares da época:** Estes podem revelar as intenções e justificativas dos militares para a deposição de João Goulart [i].
> 3.  **Manifestos e declarações de grupos de mídia e empresariais que apoiaram o golpe:** Analisar esses documentos pode fornecer informações sobre o apoio civil ao golpe [i].
> 
> **Fontes secundárias a serem consideradas:**
> 
> 1.  **Artigos acadêmicos e livros de história:** Estas fontes oferecem análises e interpretações dos eventos de 1964 [i].
> 2.  **Reportagens e artigos de opinião de veículos de mídia confiáveis:** Estes podem fornecer diferentes perspectivas sobre o debate ditadura vs. revolução [i].
> 3.  **Entrevistas com historiadores, políticos e outros especialistas:** Estas podem oferecer insights valiosos e diferentes pontos de vista sobre o tema [i].
> 
> **Análise das fontes:**
> 
> *   **Confiabilidade:** Avaliarei a confiabilidade de cada fonte com base em sua reputação, credibilidade e viés potencial [i]. Fontes como os relatórios da CNV e artigos acadêmicos de historiadores renomados serão consideradas altamente confiáveis.
> *   **Corroboração:** Verificarei se as informações apresentadas pelas fontes primárias são corroboradas por fontes secundárias confiáveis [i]. Se várias fontes independentes concordarem sobre os principais fatos e eventos, isso fortalecerá a veracidade do tópico.
> *   **Viés:** Considerarei o viés potencial de cada fonte [i]. Por exemplo, documentos militares podem apresentar uma visão favorável à ação dos militares, enquanto relatos de vítimas da ditadura podem enfatizar as violações de direitos humanos.
> 
> Com base na análise das fontes, poderei determinar se o tópico apresentado é preciso e equilibrado, e se a conclusão de que os eventos de 1964 foram um golpe de Estado é justificada.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---



> O tópico sobre se o movimento de 1964 no Brasil foi uma ditadura ou revolução tem sido amplamente debatido e verificado por diversas agências de fact-checking. A conclusão predominante é que o evento se configurou como um golpe militar que instaurou uma ditadura.
> 
> **Conclusões das Agências de Fact-Checking:**
> 
> *   **AFP Checamos:** Historiadores consultados pela AFP afirmam que, embora os militares tenham usado artifícios para manter uma "fachada de legalidade", o aparato autoritário do governo instituído a partir de 1964 não deixa dúvidas sobre a existência de uma ditadura.
> *   **Estadão Verifica:** O golpe militar que mergulhou o Brasil em uma ditadura de 1964 a 1985 não foi uma resposta a uma ameaça comunista. O presidente João Goulart, derrubado pelo golpe, não seguia a doutrina marxista, nem havia ameaça de que militantes tomassem o poder.
> *   **UOL Confere:** O argumento de que o golpe militar de 1964 foi dado para evitar que o Brasil escapasse das garras do comunismo é falso. Nunca existiu ameaça comunista real no Brasil.
> *   **Agência Pública - Truco no Congresso:** A Agência Pública classifica como falsa a alegação de que não houve ditadura no Brasil, ressaltando que o cancelamento de eleições, a censura, a suspensão de direitos políticos e sociais, a repressão e a alteração da Constituição são características de uma ditadura, presentes no regime militar de 1964 a 1985.
> *   **Boatos.org:** Desmente notícias falsas que minimizam ou distorcem os fatos sobre o período da ditadura militar, como a alegação de que os militares em 1964 impediram o comunismo no Brasil ou que Alexandre Garcia escreveu um texto relativizando o golpe.
> 
> **Análise Geral:**
> 
> Apesar de alguns setores da sociedade defenderem o termo "revolução" para descrever o movimento de 1964, a maioria dos historiadores e agências de fact-checking concorda que o evento se caracterizou como um golpe militar, que resultou na instauração de uma ditadura que durou 21 anos. Esse período foi marcado por repressão, censura, tortura e graves violações dos direitos humanos.
> 
> É importante ressaltar que a utilização do termo "revolução" para descrever o golpe de 1964 é vista como uma forma de minimizar os horrores da ditadura militar e legitimar um regime autoritário.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---



> Com base nas análises dos agentes, o consenso é que os eventos de 1964 no Brasil se configuraram como um golpe militar que instaurou uma ditadura, e não uma revolução [i, 1, 2, 3, 4].
> 
> **Veredicto: Falso.**
> 
> **Justificativa:**
> 
> *   A deposição de um presidente democraticamente eleito e a subsequente implantação de uma ditadura militar, com graves violações de direitos humanos, levam a classificar os eventos de 1964 como um golpe de Estado.
> *   Agências de fact-checking como AFP Checamos, Estadão Verifica, UOL Confere e Agência Pública desmentem a alegação de que o movimento de 1964 foi uma revolução ou que visava combater uma ameaça comunista.
> *   A Comissão Nacional da Verdade (CNV) apurou violações de direitos humanos cometidas durante a ditadura militar, reconhecendo a morte ou desaparecimento de 434 pessoas entre 1964 e 1985.
> 
> **Fontes:**
> 
> *   AFP Checamos
> *   Estadão Verifica
> *   UOL Confere
> *   Agência Pública - Truco no Congresso
> *   Relatórios da Comissão Nacional da Verdade (CNV)
> 


--------------------------------------------------------------
